### Подключение библиотек, которые понадобятся для обучения сети

In [20]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from keras.layers import SimpleRNN
from keras.layers import GRU
from keras.layers import Bidirectional

from sklearn.model_selection import GridSearchCV

from keras.wrappers.scikit_learn import KerasClassifier

from matplotlib import pyplot

### Создание функций для единовременного извлечения датасета из различных txt файлов

In [2]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

In [3]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded

In [4]:
 # load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

In [5]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

### Загрузка датасета

In [6]:
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


Формирование валидационной выборки в соотношении 2:6 к тренировочной

In [7]:
validX = trainX[:1838]
validy = trainy[:1838]

trainX = trainX[1838:]
trainy = trainy[1838:]

### Функция эксперимета, в которой будет происходить подбор гиперпараметров

In [27]:
def experiment(model, param_grid):
    grid = GridSearchCV(estimator=model, param_grid=param_grid)
    grid_result = grid.fit(validX, validy)

    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

### Подбор гиперпараметров (layers_count, nerouns_count, optimizer)на основе валидационной выборки для каждой модели (SimpleRNN, LSTM, GRU)

SimpleRNN

In [8]:
def create_SimpleRNN(neurons_count=100, layers_count=1, activation='relu', optimizer='SGD'):
    
    n_timesteps, n_features, n_outputs = validX.shape[1], validX.shape[2], validy.shape[1]
    model = Sequential()
    # Входной слой
    model.add(SimpleRNN(100, input_shape=(n_timesteps,n_features)))
    # Скрытый слой
    for _ in range (layers_count):
        model.add(Dense(neurons_count, activation=activation))
    # Выходной слой
    model.add(Dense(n_outputs, activation=activation))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [10]:
model = KerasClassifier(build_fn=create_SimpleRNN, epochs=1, batch_size=100, verbose=0)

layers_count = [1, 2, 3, 4, 5]
neurons_count = [40, 60, 80, 100, 120]
optimizer = ['SGD']
activation = ['relu']


param_grid = dict(neurons_count=neurons_count, 
                  layers_count=layers_count, 
                  activation=activation, 
                  optimizer=optimizer)

experiment(model, param_grid)

Best: 0.381954 using {'activation': 'relu', 'layers_count': 2, 'neurons_count': 100, 'optimizer': 'SGD'}
0.303519 (0.118598) with: {'activation': 'relu', 'layers_count': 1, 'neurons_count': 40, 'optimizer': 'SGD'}
0.245344 (0.133964) with: {'activation': 'relu', 'layers_count': 1, 'neurons_count': 60, 'optimizer': 'SGD'}
0.225164 (0.090571) with: {'activation': 'relu', 'layers_count': 1, 'neurons_count': 80, 'optimizer': 'SGD'}
0.365636 (0.035603) with: {'activation': 'relu', 'layers_count': 1, 'neurons_count': 100, 'optimizer': 'SGD'}
0.196886 (0.123149) with: {'activation': 'relu', 'layers_count': 1, 'neurons_count': 120, 'optimizer': 'SGD'}
0.327070 (0.112536) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 40, 'optimizer': 'SGD'}
0.301885 (0.127649) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 60, 'optimizer': 'SGD'}
0.200164 (0.065380) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 80, 'optimizer': 'SGD'}
0.381954 (0.132856) w

In [11]:
model = KerasClassifier(build_fn=create_SimpleRNN, epochs=1, batch_size=100, verbose=0)

layers_count = [2]
neurons_count = [100]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
activation = ['relu']

param_grid = dict(neurons_count=neurons_count, 
                  layers_count=layers_count, 
                  activation=activation, 
                  optimizer=optimizer)

experiment(model, param_grid)

Best: 0.384560 using {'activation': 'relu', 'layers_count': 2, 'neurons_count': 100, 'optimizer': 'Nadam'}
0.278541 (0.113183) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 100, 'optimizer': 'SGD'}
0.313360 (0.104754) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 100, 'optimizer': 'RMSprop'}
0.317796 (0.074516) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 100, 'optimizer': 'Adagrad'}
0.185498 (0.049525) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 100, 'optimizer': 'Adadelta'}
0.330826 (0.126667) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 100, 'optimizer': 'Adam'}
0.287313 (0.106092) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 100, 'optimizer': 'Adamax'}
0.384560 (0.131613) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 100, 'optimizer': 'Nadam'}


### Simple RNN. Лучший вариант гиперпараметров: 2 скрытых слоя, 100 нейронов на них, алгоритм оптимизации - Nadam

LSTM

In [12]:
def create_LSTM(neurons_count=100, layers_count=1, activation='relu', optimizer='SGD'):
    n_timesteps, n_features, n_outputs = validX.shape[1], validX.shape[2], validy.shape[1]
    
    model = Sequential()
    # Входной слой
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    # Скрытый слой
    for _ in range (layers_count):
        model.add(Dense(neurons_count, activation=activation))
    # Выходной слой
    model.add(Dense(n_outputs, activation=activation))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [13]:
model = KerasClassifier(build_fn=create_LSTM, epochs=1, batch_size=100, verbose=0)

layers_count = [1, 2, 3, 4, 5]
neurons_count = [40, 60, 80, 100, 120]
optimizer = ['SGD']
activation = ['relu']


param_grid = dict(neurons_count=neurons_count, 
                  layers_count=layers_count, 
                  activation=activation, 
                  optimizer=optimizer)

experiment(model, param_grid)

Best: 0.365078 using {'activation': 'relu', 'layers_count': 2, 'neurons_count': 40, 'optimizer': 'SGD'}
0.273553 (0.116603) with: {'activation': 'relu', 'layers_count': 1, 'neurons_count': 40, 'optimizer': 'SGD'}
0.225817 (0.087444) with: {'activation': 'relu', 'layers_count': 1, 'neurons_count': 60, 'optimizer': 'SGD'}
0.250225 (0.075433) with: {'activation': 'relu', 'layers_count': 1, 'neurons_count': 80, 'optimizer': 'SGD'}
0.243118 (0.096257) with: {'activation': 'relu', 'layers_count': 1, 'neurons_count': 100, 'optimizer': 'SGD'}
0.257409 (0.060132) with: {'activation': 'relu', 'layers_count': 1, 'neurons_count': 120, 'optimizer': 'SGD'}
0.365078 (0.061481) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 40, 'optimizer': 'SGD'}
0.263751 (0.106451) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 60, 'optimizer': 'SGD'}
0.268238 (0.146216) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 80, 'optimizer': 'SGD'}
0.210496 (0.073715) wi

In [14]:
model = KerasClassifier(build_fn=create_LSTM, epochs=1, batch_size=100, verbose=0)

layers_count = [2]
neurons_count = [40]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
activation = ['relu']

param_grid = dict(neurons_count=neurons_count, 
                  layers_count=layers_count, 
                  activation=activation, 
                  optimizer=optimizer)

experiment(model, param_grid)

Best: 0.384614 using {'activation': 'relu', 'layers_count': 2, 'neurons_count': 40, 'optimizer': 'Nadam'}
0.252035 (0.123983) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 40, 'optimizer': 'SGD'}
0.366752 (0.153756) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 40, 'optimizer': 'RMSprop'}
0.230676 (0.061277) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 40, 'optimizer': 'Adagrad'}
0.201263 (0.037152) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 40, 'optimizer': 'Adadelta'}
0.242711 (0.092176) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 40, 'optimizer': 'Adam'}
0.375382 (0.092309) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 40, 'optimizer': 'Adamax'}
0.384614 (0.077787) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 40, 'optimizer': 'Nadam'}


### LSTM. Лучший вариант гиперпараметров: 2 скрытых слоя, 40 нейронов на них, алгоритм оптимизации - Nadam

GRU

In [16]:
def create_GRU(neurons_count=100, layers_count=1, activation='relu', optimizer='SGD'):
    n_timesteps, n_features, n_outputs = validX.shape[1], validX.shape[2], validy.shape[1]
    
    model = Sequential()
    # Входной слой
    model.add(GRU(100, input_shape=(n_timesteps,n_features)))
    # Скрытый слой
    for _ in range (layers_count):
        model.add(Dense(neurons_count, activation=activation))
    # Выходной слой
    model.add(Dense(n_outputs, activation=activation))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [17]:
model = KerasClassifier(build_fn=create_GRU, epochs=1, batch_size=100, verbose=0)

layers_count = [1, 2, 3, 4, 5]
neurons_count = [40, 60, 80, 100, 120]
optimizer = ['SGD']
activation = ['relu']

param_grid = dict(neurons_count=neurons_count, 
                  layers_count=layers_count, 
                  activation=activation, 
                  optimizer=optimizer)

experiment(model, param_grid)

Best: 0.303751 using {'activation': 'relu', 'layers_count': 3, 'neurons_count': 60, 'optimizer': 'SGD'}
0.219728 (0.149893) with: {'activation': 'relu', 'layers_count': 1, 'neurons_count': 40, 'optimizer': 'SGD'}
0.234983 (0.133717) with: {'activation': 'relu', 'layers_count': 1, 'neurons_count': 60, 'optimizer': 'SGD'}
0.162664 (0.033598) with: {'activation': 'relu', 'layers_count': 1, 'neurons_count': 80, 'optimizer': 'SGD'}
0.275942 (0.089496) with: {'activation': 'relu', 'layers_count': 1, 'neurons_count': 100, 'optimizer': 'SGD'}
0.237121 (0.088286) with: {'activation': 'relu', 'layers_count': 1, 'neurons_count': 120, 'optimizer': 'SGD'}
0.175209 (0.042302) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 40, 'optimizer': 'SGD'}
0.226780 (0.089368) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 60, 'optimizer': 'SGD'}
0.299177 (0.154798) with: {'activation': 'relu', 'layers_count': 2, 'neurons_count': 80, 'optimizer': 'SGD'}
0.250271 (0.092966) wi

In [19]:
model = KerasClassifier(build_fn=create_GRU, epochs=2, batch_size=100, verbose=0)

layers_count = [3]
neurons_count = [60]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
activation = ['relu']


param_grid = dict(neurons_count=neurons_count, 
                  layers_count=layers_count, 
                  activation=activation, 
                  optimizer=optimizer)

experiment(model, param_grid)

Best: 0.463375 using {'activation': 'relu', 'layers_count': 3, 'neurons_count': 60, 'optimizer': 'Nadam'}
0.205623 (0.062595) with: {'activation': 'relu', 'layers_count': 3, 'neurons_count': 60, 'optimizer': 'SGD'}
0.369377 (0.182283) with: {'activation': 'relu', 'layers_count': 3, 'neurons_count': 60, 'optimizer': 'RMSprop'}
0.427020 (0.081613) with: {'activation': 'relu', 'layers_count': 3, 'neurons_count': 60, 'optimizer': 'Adagrad'}
0.142046 (0.059440) with: {'activation': 'relu', 'layers_count': 3, 'neurons_count': 60, 'optimizer': 'Adadelta'}
0.259574 (0.108088) with: {'activation': 'relu', 'layers_count': 3, 'neurons_count': 60, 'optimizer': 'Adam'}
0.374873 (0.184397) with: {'activation': 'relu', 'layers_count': 3, 'neurons_count': 60, 'optimizer': 'Adamax'}
0.463375 (0.166172) with: {'activation': 'relu', 'layers_count': 3, 'neurons_count': 60, 'optimizer': 'Nadam'}


### GRU. Лучший вариант гиперпараметров: 3 скрытых слоя, 60 нейронов на них, алгоритм оптимизации - Nadam

### Обучение и проверка точности на тестовой выборке

In [46]:
from tensorflow.keras import regularizers
from keras.callbacks import ReduceLROnPlateau

SimpleRNN. layers_count=2, neurons_count=100, optimizer=nadam, dropout_rate=0.4, regularizer=l1

In [73]:
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
model = Sequential()
# Входной слой
model.add(SimpleRNN(100, input_shape=(n_timesteps,n_features), kernel_regularizer='l1'))
model.add(Dropout(0.4))
# Скрытый слой
for _ in range (2):
    model.add(Dense(100, activation='relu'))
# Выходной слой
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=3, min_lr=0.001)

for _ in range(10):
    model.fit(trainX, trainy, epochs=10, batch_size=64, verbose=2, callbacks=[reduce_lr])
    _, accuracy = model.evaluate(testX, testy, batch_size=64)
    if accuracy >= 0.91:
        break

Epoch 1/10
87/87 - 5s - loss: 2.0167 - accuracy: 0.5176
Epoch 2/10
87/87 - 4s - loss: 1.4008 - accuracy: 0.6306
Epoch 3/10
87/87 - 4s - loss: 1.0859 - accuracy: 0.6852
Epoch 4/10
87/87 - 4s - loss: 1.2219 - accuracy: 0.6422
Epoch 5/10
87/87 - 4s - loss: 1.3600 - accuracy: 0.6146
Epoch 6/10
87/87 - 4s - loss: 1.5780 - accuracy: 0.4527
Epoch 7/10
87/87 - 4s - loss: 1.4553 - accuracy: 0.4704
Epoch 8/10
87/87 - 4s - loss: 1.3851 - accuracy: 0.4913
Epoch 9/10
87/87 - 4s - loss: 1.3231 - accuracy: 0.5176
Epoch 10/10
87/87 - 4s - loss: 1.2109 - accuracy: 0.5769
47/47 [==============================] - 1s 23ms/step - loss: 1.6331 - accuracy: 0.3590
Epoch 1/10
87/87 - 4s - loss: 1.4342 - accuracy: 0.4735
Epoch 2/10
87/87 - 4s - loss: 1.2629 - accuracy: 0.5453
Epoch 3/10
87/87 - 4s - loss: 1.1567 - accuracy: 0.6055
Epoch 4/10
87/87 - 4s - loss: 1.0610 - accuracy: 0.6340
Epoch 5/10
87/87 - 4s - loss: 1.0531 - accuracy: 0.6427
Epoch 6/10
87/87 - 4s - loss: 2.0598 - accuracy: 0.2896
Epoch 7/10
87/8

SimpleRNN не оправдала надежд. За 100 эпох точность на тестовой выборке не достигла порогового значения

LSTM. layers_count=2, neurons_count=40, optimizer=nadam, dropout_rate=0.4, regularizer=l1

In [74]:
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
model = Sequential()
# Входной слой
model.add(LSTM(100, input_shape=(n_timesteps,n_features), kernel_regularizer='l1'))
model.add(Dropout(0.4))
# Скрытый слой
for _ in range (2):
    model.add(Dense(40, activation='relu'))
# Выходной слой
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=3, min_lr=0.001)


for _ in range(10):
    model.fit(trainX, trainy, epochs=10, batch_size=64, verbose=2, callbacks=[reduce_lr])
    _, accuracy = model.evaluate(testX, testy, batch_size=64)
    if accuracy >= 0.91:
        break

Epoch 1/10
87/87 - 11s - loss: 2.7534 - accuracy: 0.3357
Epoch 2/10
87/87 - 10s - loss: 1.5158 - accuracy: 0.4554
Epoch 3/10
87/87 - 10s - loss: 1.2973 - accuracy: 0.5053
Epoch 4/10
87/87 - 10s - loss: 1.1853 - accuracy: 0.5413
Epoch 5/10
87/87 - 9s - loss: 1.1567 - accuracy: 0.5450
Epoch 6/10
87/87 - 9s - loss: 1.1091 - accuracy: 0.5716
Epoch 7/10
87/87 - 10s - loss: 1.0523 - accuracy: 0.6114
Epoch 8/10
87/87 - 10s - loss: 1.2271 - accuracy: 0.5593
Epoch 9/10
87/87 - 10s - loss: 0.9676 - accuracy: 0.6395
Epoch 10/10
87/87 - 10s - loss: 0.8144 - accuracy: 0.6841
47/47 [==============================] - 3s 53ms/step - loss: 1.1073 - accuracy: 0.5684
Epoch 1/10
87/87 - 10s - loss: 0.7815 - accuracy: 0.7111
Epoch 2/10
87/87 - 9s - loss: 0.7421 - accuracy: 0.7490
Epoch 3/10
87/87 - 10s - loss: 0.5921 - accuracy: 0.7954
Epoch 4/10
87/87 - 10s - loss: 0.5501 - accuracy: 0.8342
Epoch 5/10
87/87 - 10s - loss: 0.5725 - accuracy: 0.8384
Epoch 6/10
87/87 - 10s - loss: 0.7834 - accuracy: 0.7891
Ep

Модель LSTM достигла порогового значения точности на тестовой выборке.

GRU. layers_count=3, neurons_count=60, optimizer=nadam, dropout_rate=0.4, regularizer=l1

In [75]:
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
model = Sequential()
# Входной слой
model.add(GRU(100, input_shape=(n_timesteps,n_features), kernel_regularizer='l1'))
model.add(Dropout(0.4))
# Скрытый слой
for _ in range (3):
    model.add(Dense(60, activation='relu'))
# Выходной слой
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=3, min_lr=0.001)


for _ in range(10):
    model.fit(trainX, trainy, epochs=10, batch_size=64, verbose=2, callbacks=[reduce_lr])
    _, accuracy = model.evaluate(testX, testy, batch_size=64)
    if accuracy >= 0.91:
        break

Epoch 1/10
87/87 - 9s - loss: 2.3839 - accuracy: 0.4565
Epoch 2/10
87/87 - 10s - loss: 1.2674 - accuracy: 0.5225
Epoch 3/10
87/87 - 8s - loss: 1.1431 - accuracy: 0.5461
Epoch 4/10
87/87 - 8s - loss: 1.0860 - accuracy: 0.5608
Epoch 5/10
87/87 - 8s - loss: 1.0379 - accuracy: 0.5903
Epoch 6/10
87/87 - 9s - loss: 0.9936 - accuracy: 0.6453
Epoch 7/10
87/87 - 8s - loss: 1.4719 - accuracy: 0.4862
Epoch 8/10
87/87 - 8s - loss: 1.4193 - accuracy: 0.4786
Epoch 9/10
87/87 - 9s - loss: 1.2878 - accuracy: 0.5205
Epoch 10/10
87/87 - 10s - loss: 1.2169 - accuracy: 0.5413
47/47 [==============================] - 2s 38ms/step - loss: 1.2752 - accuracy: 0.4849
Epoch 1/10
87/87 - 11s - loss: 1.1955 - accuracy: 0.5515
Epoch 2/10
87/87 - 12s - loss: 1.2075 - accuracy: 0.5426
Epoch 3/10
87/87 - 11s - loss: 1.4414 - accuracy: 0.4648
Epoch 4/10
87/87 - 10s - loss: 1.3850 - accuracy: 0.4159
Epoch 5/10
87/87 - 11s - loss: 1.3561 - accuracy: 0.4189
Epoch 6/10
87/87 - 14s - loss: 1.3358 - accuracy: 0.4324
Epoch 7

Модель на основе GRU почти достигла порогового значения за 100 эпох, но так и не превзошла его.

Запуск модели LSTM без early stopping для замера конечной точности. 150 эпох

In [76]:
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
model = Sequential()
# Входной слой
model.add(LSTM(100, input_shape=(n_timesteps,n_features), kernel_regularizer='l1'))
model.add(Dropout(0.4))
# Скрытый слой
for _ in range (2):
    model.add(Dense(40, activation='relu'))
# Выходной слой
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=3, min_lr=0.001)


for _ in range(15):
    model.fit(trainX, trainy, epochs=10, batch_size=64, verbose=2, callbacks=[reduce_lr])
    _, accuracy = model.evaluate(testX, testy, batch_size=64)

Epoch 1/10
87/87 - 10s - loss: 2.7388 - accuracy: 0.3197
Epoch 2/10
87/87 - 8s - loss: 1.4844 - accuracy: 0.3698
Epoch 3/10
87/87 - 9s - loss: 1.4049 - accuracy: 0.3921
Epoch 4/10
87/87 - 10s - loss: 1.3668 - accuracy: 0.4246
Epoch 5/10
87/87 - 9s - loss: 1.5299 - accuracy: 0.4008
Epoch 6/10
87/87 - 10s - loss: 1.3799 - accuracy: 0.4287
Epoch 7/10
87/87 - 9s - loss: 1.1842 - accuracy: 0.5317
Epoch 8/10
87/87 - 9s - loss: 1.0009 - accuracy: 0.6025
Epoch 9/10
87/87 - 9s - loss: 0.7837 - accuracy: 0.6993
Epoch 10/10
87/87 - 10s - loss: 0.6708 - accuracy: 0.7490
47/47 [==============================] - 4s 81ms/step - loss: 0.8972 - accuracy: 0.6407
Epoch 1/10
87/87 - 10s - loss: 0.6166 - accuracy: 0.7855
Epoch 2/10
87/87 - 10s - loss: 0.5325 - accuracy: 0.8255
Epoch 3/10
87/87 - 10s - loss: 0.4893 - accuracy: 0.8509
Epoch 4/10
87/87 - 10s - loss: 0.5385 - accuracy: 0.8484
Epoch 5/10
87/87 - 10s - loss: 0.5221 - accuracy: 0.8429
Epoch 6/10
87/87 - 10s - loss: 0.4280 - accuracy: 0.8803
Epoch

Epoch 6/10
87/87 - 16s - loss: 0.2673 - accuracy: 0.9278
Epoch 7/10
87/87 - 16s - loss: 0.1861 - accuracy: 0.9461
Epoch 8/10
87/87 - 16s - loss: 0.1726 - accuracy: 0.9505
Epoch 9/10
87/87 - 16s - loss: 0.3404 - accuracy: 0.9249
Epoch 10/10
87/87 - 15s - loss: 1.2850 - accuracy: 0.6224
47/47 [==============================] - 4s 90ms/step - loss: 1.7407 - accuracy: 0.5019  - ETA: 0s - loss: 1.7710 - ac
Epoch 1/10
87/87 - 16s - loss: 1.7610 - accuracy: 0.3683
Epoch 2/10
87/87 - 16s - loss: 1.4483 - accuracy: 0.4496
Epoch 3/10
87/87 - 16s - loss: 1.3957 - accuracy: 0.4623
Epoch 4/10
87/87 - 16s - loss: 1.2141 - accuracy: 0.5403
Epoch 5/10
87/87 - 16s - loss: 1.0736 - accuracy: 0.5985
Epoch 6/10
87/87 - 16s - loss: 0.8519 - accuracy: 0.6476
Epoch 7/10
87/87 - 15s - loss: 0.7635 - accuracy: 0.6870
Epoch 8/10
87/87 - 14s - loss: 0.7939 - accuracy: 0.6950
Epoch 9/10
87/87 - 16s - loss: 0.6100 - accuracy: 0.7731
Epoch 10/10
87/87 - 16s - loss: 0.5238 - accuracy: 0.8201
47/47 [=================

Максимальное значение точности для данной модели которое было зафиксированно равно 0.9186